<a href="https://colab.research.google.com/github/MaximTislenko/DS_in_MED/blob/main/Less_03_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import requests
from bs4 import BeautifulSoup
import re
import os

#Работа с PUG-REST

Попробуем получить все данные по аспирину

In [39]:
data_p = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/aspirin/xml")
xml = BeautifulSoup(data_p.content, 'xml')
SM_tag = xml.find(name="PC-Urn_name", string="Canonical")
SM_parents = SM_tag.find_parent("PC-InfoData")
# SM_parents
X = SM_parents.find('PC-InfoData_value_sval').string
X

'CC(=O)OC1=CC=CC=C1C(=O)O'

Скрипт для очистки данных о температуре плавления, полученных от PubChem.
##НАДО РАЗБИРАТЬСЯ!!!

In [ ]:
if not os.path.exists('/content/gdrive/MyDrive/Data/mp/clean.csv'):
  data['mp'] = None

  # discard data with spread beyond this tolerance level/отбрасывать данные с разбросом, превышающим этот уровень допуска
  tolerance = 1

  for key in melting_point:
    # Get index to later input data/Получить индекс для последующих входных данных
    index = data[data['cid'] == key].index

    # Get all the melting points of a compound/Получить все точки плавления соединения
    values = melting_point[key]

    farenheit = []
    celcius = []

    # Get strings that have farenheit/celcius and ignore those with annotations
    # Получить строки с градусами Фаренгейта/Цельсия и игнорировать строки с аннотациями.
    for value in values:
      temp = value.get_text()
      if bool(re.search("^[\-|0-9][0-9]*[.]?[0-9]*[ ]*[u'\N{DEGREE SIGN}'][F ]$", temp)):

        temp = temp.split(u'\N{DEGREE SIGN}')[0].strip()
        farenheit.append(float(temp))

      elif bool(re.search("^[\-|0-9][0-9]*[.]?[0-9]*[ ]*[u'\N{DEGREE SIGN}'][C ]$", temp)):

        temp = temp.split(u'\N{DEGREE SIGN}')[0].strip()
        celcius.append(float(temp))

    # Situation 1: reports with both T scale/Ситуация 1: отчеты с обеими T-шкалами
    if farenheit and celcius:
      if np.std(farenheit) < tolerance and np.std(celcius) < tolerance:
        if abs(f_to_c(np.mean(farenheit)) - np.mean(celcius)) < 1:

          data.loc[index, 'mp'] = round(np.mean(celcius), 2)

    # Situation 2: reports with only farenheit/Ситуация 2: отчеты содержат только фаренгейты
    elif farenheit and not celcius:
      if np.std(farenheit) < tolerance:

        data.loc[index, 'mp'] = f_to_c(np.mean(farenheit))

    # Situation 3: reports with only celcius/Ситуация 3: отчеты содержат только градусы Цельсия
    elif celcius and not farenheit:
      if np.std(celcius) < tolerance:

        data.loc[index, 'mp'] = round(np.mean(celcius), 2)

  data = data.dropna().reset_index(drop=True)
  data['mp'] = data['mp'].astype('float64')
  data.to_csv('/content/gdrive/MyDrive/Data/mp/clean.csv', index=False)
else:
  data = pd.read_csv('/content/gdrive/MyDrive/Data/mp/clean.csv')

TypeError: ignored